# 개념 문제 (ㅇㅇ / ㄴㄴ)

## 이곳은 개념을 복습하는 곳입니다 





### 1. SGD(Stochasitc Gradient Descent) 를 이용하면 null 값이 많은 행렬을 유추해 낼 수 있다 (O, X)

### 2. RMSE(Root Mean Square Deviation)을 이용해 유추한 행렬이 실제값과 얼마나 차이나는지 확인할 수 있다 (O,X)

### 3. Scikit의 Surprise 패키지는 자주 사용하는 핵심 UI와 이름이 달라 사용하기 불편하다 (O,X)










# 실습을 통해 확인해보겠습니다. 


## 준비물 :

### grouplens.org/datasets/movielens/latest 에서 1mb 데이터 세트 


### pip install scikit-surprise


# 다음은 행렬 분해를 위한 사용자 정의 함수입니다.

In [4]:
#확률적 경사 하강법을 이용한 행렬 분해

import numpy as np
from sklearn.metrics import mean_squared_error

def get_rmse(R, P, Q, non_zeros):
    error = 0
    full_pred_matrix = np.dot(P, Q.T)
    
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)
    
    return rmse


# 문제 1 : 다음의 get_rmse 함수는 어떠한 기능을 하나요? 

### 정답 >>




### --------------------------------------------------------------

# 행렬 분해 함수

In [3]:
def matrix_factorization(R, K , steps = 200, learning_rate = 0.01, r_lambda = 0.01):
    num_users, num_items = R.shape
    np.random.seed(1)
    P = np.random.normal(scale = 1./K, size = (num_users, K))
    Q = np.random.normal(scale = 1./K, size = (num_items, K))
    
    prev_rmse = 10000
    break_count = 0
    
    non_zeros = [(i,j,R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j]>0]
    
    for step in range(steps):
        for i, j, r in non_zeros:
            eij = r - np.dot(P[i,:], Q[j, :].T)
            P[i,:] = P[i,:] + learning_rate*(eij * Q[j,:] - r_lambda*P[i,:])
            Q[j,:] = Q[j,:] + learning_rate*(eij * P[i,:] - r_lambda*Q[j,:])
            
        rmse = get_rmse(R,P,Q, non_zeros)
        if (step%10) == 0:
            print('### 몇번째 진행중이냐면: ', step, '실제값과의 오차는(rmse) :', rmse)
            
    return P, Q

## 필요 파일 불러오기

In [5]:
import pandas as pd
import numpy as np

import pandas as pd
import numpy as np

movies = pd.read_csv('dataset/movies.csv')
ratings = pd.read_csv('dataset/ratings.csv')
ratings = ratings[['userId', 'movieId', 'rating']]
ratings_matrix = ratings.pivot_table('rating', index='userId', columns='movieId')

rating_movies = pd.merge(ratings, movies, on='movieId')

ratings_matrix = rating_movies.pivot_table('rating', index='userId', columns='title')
ratings_matrix.head()

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 문제 2. 

## 저는 userId가 4인 사람에게 Rating 예측에 기반해 영화 탑 3를 추천하고 싶습니다. 

## 2 - 1 해당 영화의 제목을 출력하는 코드를 작성하세요.
#### => 


## 2 - 2 영화 3편이 무엇인지 알려주세요.

#### => 



### 해당 코드를 참고하세요


In [ ]:
P, Q = matrix_factorization(ratings_matrix.values, K=50, steps=50, learning_rate=0.01, r_lambda = 0.01)
pred_matrix = np.dot(P, Q.T)



In [ ]:
ratings_pred_matrix = pd.DataFrame(data=pred_matrix, index= ratings_matrix.index,
                                   columns = ratings_matrix.columns)

ratings_pred_matrix.head(3)

In [ ]:
def get_unseen_movies(ratings_matrix, userId):
    # userId로 입력받은 사용자의 모든 영화정보 추출하여 Series로 반환함. 
    # 반환된 user_rating 은 영화명(title)을 index로 가지는 Series 객체임. 
    user_rating = ratings_matrix.loc[userId,:]
    
    # user_rating이 0보다 크면 기존에 관람한 영화임. 대상 index를 추출하여 list 객체로 만듬
    already_seen = user_rating[ user_rating > 0].index.tolist()
    
    # 모든 영화명을 list 객체로 만듬. 
    movies_list = ratings_matrix.columns.tolist()
    
    # list comprehension으로 already_seen에 해당하는 movie는 movies_list에서 제외함. 
    unseen_list = [ movie for movie in movies_list if movie not in already_seen]
    
    return unseen_list


In [ ]:
def recomm_movie_by_userid(pred_df, userId, unseen_list, top_n=10):
    # 예측 평점 DataFrame에서 사용자id index와 unseen_list로 들어온 영화명 컬럼을 추출하여
    # 가장 예측 평점이 높은 순으로 정렬함. 
    recomm_movies = pred_df.loc[userId, unseen_list].sort_values(ascending=False)[:top_n]
    return recomm_movies

In [ ]:
userId =24;

unseen_list = get_unseen_movies(ratings_matrix, userId)

recomm_movies = recomm_movie_by_userid(ratings_pred_matrix, userId, unseen_list, top_n=10)
 
recomm_movies = pd.DataFrame(data=recomm_movies.values,index=recomm_movies.index,columns=['pred_score'])
recomm_movies

# 다음은 Surprise 패키지를 사용하여 추천 시스템을 구축하는 것입니다. 


### 필요한 것 import


In [7]:
import surprise 
from surprise import SVD
from surprise import Dataset 
from surprise import accuracy 
from surprise.model_selection import train_test_split
print(surprise.__version__)


1.1.0


# Suprise 에서 추천을 예측하는 메서드는 test() 와 predict() 가 있습니다 

## 첫번째로 test를 이용해 데이터세트 전체 추천 데이터 세트를 만들어 봅시다!

In [84]:
data = Dataset.load_builtin('ml-100k') 
trainset, testset = train_test_split(data, test_size=.25, random_state=0)
# random state 0 으로 꼭 해주시기 바랍니다!!

#### SVD()를 이용해 객체 선언 후 fit() 을 이용합니다. 
#### fit() 어디에서 많이 본 함수입니다!!

In [9]:
algo = SVD()
algo.fit(trainset)

#### 입력한 데이터셋 전체에 대한 예측 데이터를 생성합니다! (여기서는 testset)

In [10]:
predictions = algo.test( testset )
print('prediction type :',type(predictions), ' size:',len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

prediction type : <class 'list'>  size: 25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.538081629620365, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=4.105070064358905, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.203674770073809, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.742931900834871, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.456751127893282, details={'was_impossible': False})]

## 문제 1. 사용자ID 120 의 영화ID 282에 대한 rating 예측 값을 출력하세요

In [92]:

## 답변 :  



## 문제 2. 사용자ID 120 의 영화ID 282에 대한 rating 예측 값을 predict()를 통해 출력하세요

In [93]:
## 답변 : 


